In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.set_option("display.max_rows", 300)  #表示する行数を増やす

In [ ]:
keyword = 'sumou'
period = '2018-07-08'
infile = "input/"+keyword+period+".csv"

In [ ]:
datas = pd.read_csv(open(infile, 'rU'), encoding='utf-8')

In [ ]:
datas

# botとか不適切やツイートを除く

In [ ]:
datas = datas.drop('Unnamed: 0', axis=1)

In [ ]:
improper_col = []
for col in range(datas.shape[0]):
#    if type(datas.name[col]) != str or type(datas.profile[col]) != str or type(datas.text[col]) != str:
    if type(datas.text[col]) != str:
        print('-'*40)
        print(col)
        print(type(datas.name[col]), type(datas.profile[col]), type(datas.text[col]),)
        print(datas.iloc[col])
        improper_col.append(col)
datas = datas.drop(improper_col, axis=0)
datas = datas.reset_index(drop=True)

In [ ]:
datas['n_following'] = datas['n_following'].astype('int')
datas['n_followed'] = datas['n_followed'].astype('int')
datas['n_tweets'] = datas['n_tweets'].astype('int')
datas['n_favorited'] = datas['n_favorited'].astype('int')

In [ ]:
improper_names  = ['bot', 'Bot', 'BOT', 'ぼっと', '情報', '案内', '相互', '出会', 'セフレ',
                   'エッチ', '法人', '相撲 バズウォール', '逢華', '大西啓太', 'News', 'NEWS',
                   'news', 'ニュース', '新聞', '報道', 'バカボンのパパ', '久保寺健之',
                   '沖縄タイムス', 'チケット×チケット']
improper_profiles  = ['improper_words', 'bot', 'Bot', 'BOT', 'ぼっと', '公式ツイッター', '公式アカウント']
improper_texts  = ['improper_words', "I'm at", '相互', 'imacoconow', '火ノ丸', '大喜利',
                   '手押し相撲' , '一人相撲' , '独り相撲', 'ローション', '名言集', '紙相撲',
                   '火の丸', 'カーネーション', '他人の褌で相撲を取る', '菊とギロチン', '伝令',
                   '格闘技、プロ野球、相撲好き', '女相撲', '氷結相撲', '尻相撲', 'とんとん相撲',
                   'トントン相撲', 'バブル相撲', 'ストッキング相撲', 'ちびっこ相撲']

for col in range(datas.shape[0]):
    if type(datas['name'][col]) == str:
        for word in improper_names:
            if word in datas['name'][col]:
                datas['profile'][col] = 'improper_words'
                break
    if type(datas['profile'][col]) == str:
        for word in improper_profiles:
            if word in datas['profile'][col]:
                datas['text'][col] = 'improper_words'
                break
    for word in improper_texts:
        if word in datas['text'][col]:
            datas = datas.drop(col, axis=0)
            break

datas = datas.reset_index(drop=True)

In [ ]:
datas

# EDA

In [ ]:
n_keyword_tweets = datas['id'].value_counts()
n_keyword_tweets_arr = []

for col in range(datas.shape[0]):
    n_keyword_tweets_arr.append(n_keyword_tweets[datas['id'][col]])
datas['n_keyword_tweets'] = n_keyword_tweets_arr

del n_keyword_tweets, n_keyword_tweets_arr

In [ ]:
datas_unique = datas.drop_duplicates(subset='id')
datas_unique = datas_unique.reset_index(drop=True)
datas_unique

In [ ]:
datas_unique.describe()

# キーワードを多くツイートした人

In [ ]:
sns.distplot(datas['n_keyword_tweets'])

In [ ]:
datas_unique.sort_values('n_keyword_tweets', ascending=False)

In [ ]:
datas_unique['adress'].value_counts()

In [ ]:
datas.sort_values('n_keyword_tweets', ascending=False)

In [ ]:
sns.distplot(np.log1p(datas_unique['n_following']))

In [ ]:
datas_unique.sort_values(by='n_following', ascending=False)

In [ ]:
sns.distplot(np.log1p(datas_unique['n_followed']))

In [ ]:
datas_unique.sort_values(by='n_followed', ascending=False)

In [ ]:
sns.distplot(np.log1p(datas_unique['n_tweets']))

In [ ]:
datas_unique.sort_values(by='n_tweets', ascending=False)

In [ ]:
sns.distplot(np.log1p(datas_unique['n_favorited']))

In [ ]:
sns.countplot(datas_unique['n_favorited'])

In [ ]:
datas_unique.sort_values(by='n_favorited', ascending=False)

In [ ]:
datas

# キーワードと共にツイートされることが多かった単語

In [ ]:
import MeCab
import re

m = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")
word_list = []
for i in range(datas.shape[0]):
    if type(datas.text[i]) == str:
        texts = m.parse(datas['text'][i])
        texts = texts.split('\n')
        for text in texts:
            text = re.split('[\t,]', text)
            if text[0] == 'EOS' or text[0] == '':
                pass
            elif text[1] == '名詞': 
                word_list.append(text[0])
#            elif text[1] == '名詞' and text[2] == '一般': 
#                word_list.append(text[0])
#            elif text[1] == '名詞' and text[2] == 'サ変接続': 
#                word_list.append(text[0])
#            elif text[1] == '名詞' and text[2] == '形容動詞語幹': 
#                word_list.append(text[0])
#            elif text[1] == '形容詞' and text[2] == '自立': 
#                word_list.append(text[0])

In [ ]:
from collections import Counter

counter = Counter(word_list)
for word, cnt in counter.most_common():
    print(word, cnt)

In [ ]:
stop_words = [
    'in', 'ー', 'bot', 'https', 'co', 'ない', '無い', '投稿', 'ツイート', '今日', '明日',
    'さん', 'こと', 'よう', 'それ', 'どこ', 'これ', 'みたい', '名前', '自分', 'ちゃん', 'そう',
    '登録', 'くん', 'あと', 'そこ', 'ため', 'うち', 'ここ', 'ところ', 'なん', '感じ', '情報',
    'もの', 'とき', 'やつ', 'もん', 'しよう', 'わけ', 'たち', 'とこ', 'つもり', 'こちら',
    'しんみ', 'した', 'せい', 'さま', 'さっき', 'こっち', 'かな', 'まま', '最近', '時間',
    '場所', '本日', '付近', 'よろしくお願いします', '昨日', '今週', '来週', '先週', 'みんな',
    '相撲', '名古屋場所', 'sumo', '大相撲', '力士', '中継', '相撲部', '相撲取り',
    '大相撲名古屋場所','相手', '土俵', '？？？', 'あれ', '近く', 't', 'の', 'ん', 'w',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '０', '１', '２','３', '４',
    '５', '６', '７', '８', '９', '方', '何', '中', '笑', '今', 'さ', '時', '関', '事',
    '気', 'ニュース', '話', 'www', 'ω', '的', '目', '様', '後', '俺', 'スーモ', '日',
    '手', '次', '観', '君', '…。', '山', '感', '回', 'ww', 'SD', '差'
]

def remove_specified_values(arr, value):
    while value in arr:
        arr.remove(value)

for word in stop_words:
    remove_specified_values(word_list, word)

In [ ]:
counter = Counter(word_list)
for word, cnt in counter.most_common():
    print(word, cnt)

In [ ]:
word_list = ' '.join(word_list)

In [ ]:
from wordcloud import WordCloud

fpath = "~/Library/Fonts/RictyDiminished-Regular.ttf"

wordcloud = WordCloud(background_color="white", font_path=fpath, width=900, height=500, max_words=80).generate(word_list)

plt.figure(figsize=(10,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()
#一文字の単語は表示してくれない...